# Qhue experiments

Experiments with the [Qhue](https://github.com/quentinsf/qhue) python module.

If you haven't already, then `pip install qhue` before starting.  

Some of these examples may assume you have a recent bridge with recent software.

## Basics 

In [1]:
# Put in the IP address of your Hue bridge here
BRIDGE_IP='192.168.0.45'

from qhue import Bridge, QhueException, create_new_username


In [3]:
# If you have a username set up on your bridge, enter it here
# otherwise leave it as None and you'll be prompted to create one.
# e.g.:
# username='zeZomfNu-y-p1PLM9oeYTiXbtqsxn-q1-7RNLI4B'

username=None

if username is None:
    username = create_new_username(BRIDGE_IP)
    print "New user: {} . Put this in the username variable above.".format(username)

Let's get the numbers and names of the lights:

In [4]:
bridge = Bridge(BRIDGE_IP, username)
lights = bridge.lights()
for num, info in lights.items():
    print "{:16} {}".format(info['name'], num)

Kitchen table    11
Mantlepiece R    24
Shed             23
Kitchen          12
Picture rail     15
Sitting room 2   14
Dining table 1   17
Dining table 3   16
Dining table 2   18
Bedroom 1        22
Kitchen Sink     3
Top Landing      5
Landing          4
Front hall 1     7
Kitchen Stove    6
Front hall 2     8
Mantlepiece L    25
Sitting room 1   21


Let's try interactively changing a light.  You could make this a lot more sophisticated:

In [5]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

def setlight(lightid='14', on=True, ct=128, bri=128):
    bridge.lights[lightid].state(on=on)
    if on:
        bridge.lights[lightid].state(bri=bri, ct=ct)

light_list = interact(setlight,
                      lightid = widgets.Dropdown(
                            options={ lights[i]['name']:i for i in lights },
                            value='14',
                            description='Light:',
                        ),
                      on = widgets.Checkbox(value=True, description='On/off'),
                      bri = widgets.IntSlider(min=0,max=255,value=128, description='Bright:'),
                      ct = widgets.IntSlider(min=0,max=255,value=128, description='Colour:'))

The [YAML format](https://en.wikipedia.org/wiki/YAML) is a nice way to view the sometimes large amount of structured information which comes back from the bridge. 

If you haven't got the Python yaml module, `pip install PyYAML`.

In [9]:
import yaml
print len(lights), "lights\n"
print yaml.safe_dump(lights, indent=4)

18 lights

'11':
    manufacturername: Philips
    modelid: LCT007
    name: Kitchen table
    state:
        alert: none
        bri: 254
        colormode: xy
        ct: 223
        effect: none
        hue: 33910
        'on': false
        reachable: true
        sat: 71
        xy: [0.3622, 0.3646]
    swversion: 5.38.1.14919
    type: Extended color light
    uniqueid: 00:17:88:01:00:f7:e8:58-0b
'12':
    manufacturername: Philips
    modelid: LCT007
    name: Kitchen
    state:
        alert: none
        bri: 254
        colormode: xy
        ct: 239
        effect: none
        hue: 33810
        'on': false
        reachable: true
        sat: 28
        xy: [0.3734, 0.3722]
    swversion: 5.38.1.14919
    type: Extended color light
    uniqueid: 00:17:88:01:00:f6:e4:98-0b
'14':
    manufacturername: Philips
    modelid: LCT001
    name: Sitting room 2
    state:
        alert: none
        bri: 189
        colormode: xy
        ct: 463
        effect: none
        hue: 3941

In [10]:
print yaml.safe_dump(bridge.lights['24'](), indent=4) 

manufacturername: OSRAM
modelid: Classic B40 TW - LIGHTIFY
name: Mantlepiece R
state: {alert: select, bri: 1, colormode: ct, ct: 370, 'on': false, reachable: true}
swversion: V1.03.20
type: Color temperature light
uniqueid: 84:18:26:00:00:09:59:e3-03



## Scenes 

Let's look at the scenes defined in the bridge, and their IDs.  Some of these may be created manually, and others by the Hue app or other software.

Version 1-type scenes just refer to the lights - each light is told: "Set the value you have stored for this scene".

Version 2 scenes have more details stored in the hub, which is generally more useful.

In [11]:
scenes = bridge.scenes()
print len(scenes), "scenes\n"
print yaml.safe_dump(scenes, indent=4)

133 scenes

-1iru0uZghuPUgV:
    appdata: {data: 3dS2m_r04_d03, version: 1}
    lastupdated: '2016-07-23T10:42:48'
    lights: ['3', '6', '11', '12', '15', '16', '17', '18']
    locked: false
    name: Concentrate
    owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
    picture: ''
    recycle: true
    version: 2
06869458c-on-0:
    appdata: {}
    lastupdated: null
    lights: ['4', '7', '8']
    locked: false
    name: Bright hall land
    owner: none
    picture: ''
    recycle: true
    version: 1
07981a34a-on-0:
    appdata: {}
    lastupdated: null
    lights: ['4', '5']
    locked: true
    name: 'Warm landing on '
    owner: none
    picture: ''
    recycle: true
    version: 1
0d9qlRyOk7Td5-0:
    appdata: {data: PcOcf_r07_d06, version: 1}
    lastupdated: '2016-04-29T07:54:46'
    lights: ['7', '8']
    locked: false
    name: Dimmed
    owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
    picture: ''
    recycle: false
    version: 2
0f7797c16-on-0:
    appdata: {}
    lastup

Details of a particular scene from the list:

In [53]:
print yaml.safe_dump(bridge.scenes['wVXtOrFmdnySqUz']())

appdata: {data: skbwq_r06_d06, version: 1}
lastupdated: '2016-08-06T18:25:59'
lights: ['14', '21', '24', '25']
lightstates:
  '14': {bri: 77, ct: 366, 'on': true}
  '21': {bri: 77, ct: 366, 'on': true}
  '24': {bri: 77, ct: 366, 'on': true}
  '25': {bri: 77, ct: 366, 'on': true}
locked: false
name: Dimmed
owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
picture: ''
recycle: false
version: 2



Let's list scenes with IDs, last updated time, and the lights affected:

In [12]:
for sid, info in scenes.items():
    print "\n{:16} {:20} {}".format( sid, info['name'], info['lastupdated'])
    for li in info['lights']:
        print "{:40}- {}".format('', lights[li]['name'])


6VRh7MuKX1Nheu3  Concentrate          2016-07-23T10:43:38
                                        - Landing
                                        - Top Landing

AsptFt3pCldMO21  Last on state        2016-07-23T10:43:34
                                        - Front hall 1
                                        - Front hall 2

T6UzpfPUK2kmvu7  Energize             2016-07-23T10:42:49
                                        - Kitchen Sink
                                        - Kitchen Stove
                                        - Kitchen table
                                        - Kitchen
                                        - Picture rail
                                        - Dining table 3
                                        - Dining table 1
                                        - Dining table 2

cff986e21-on-15  Spring landing o     None
                                        - Landing
                                        - Top Landing

06869458c-on-0   

Tidying things up; let's delete a scene:

In [13]:
# Uncomment and edit this if you actually want to run it!
# print bridge.scenes['cd06c70f7-on-0'](http_method='delete')

[{u'success': u'/scenes/cd06c70f7-on-0 deleted'}]


Show the details of the scenes that affect a particular light:

In [14]:
lightname = 'Sitting room 1'
# How's this for a nice use of python iterators?
light_id = next(i for i,info in lights.items() if info['name'] == lightname)
print "Light", light_id, lightname
for line in ["{} : {:20} {}".format(sid, info['name'], info['lastupdated']) for sid, info in scenes.items() if light_id in info['lights']]:
    print line

Light 21 Sitting room 1
wG25IXpcHHTim4g : Off                  2016-08-06T18:26:02
YDfVlYFWoaL6yv5 : Nightlight           2016-08-06T18:25:59
wVXtOrFmdnySqUz : Dimmed               2016-08-06T18:25:59
SePln7Lt9-H7Hm7 : Bright               2016-08-07T09:35:44
SUThT3XiV7sSzml : Warm                 2016-08-07T10:19:18
KZNM2DZmdcydRIc : All warm             2016-08-06T23:50:55
VaknVPkUZnSrdiB : Bright               2016-08-06T18:39:00
IB57U3scrj4cQWk : Read                 2016-08-06T18:25:58
3owQUn01W7nVsxR : Evening              2016-08-07T10:19:47


## Groups and rooms

Let's look at groups:

In [38]:
print yaml.safe_dump(bridge.groups(), indent=4)

'1':
    action:
        alert: none
        bri: 254
        colormode: xy
        ct: 262
        effect: none
        hue: 16635
        'on': true
        sat: 18
        xy: [0.389, 0.3819]
    lights: ['3', '6']
    name: Kitchen
    recycle: false
    state: {all_on: true, any_on: true}
    type: LightGroup
'2':
    action:
        alert: none
        bri: 144
        colormode: xy
        ct: 443
        effect: none
        hue: 13524
        'on': false
        sat: 200
        xy: [0.5017, 0.4152]
    lights: ['7', '8']
    name: Hall
    recycle: false
    state: {all_on: false, any_on: false}
    type: LightGroup
'3':
    action:
        alert: none
        bri: 254
        colormode: xy
        ct: 226
        effect: none
        hue: 33887
        'on': true
        sat: 61
        xy: [0.3649, 0.3665]
    lights: ['12', '11', '6', '3']
    name: Dimmer 11
    recycle: false
    state: {all_on: true, any_on: true}
    type: LightGroup
'4':
    action:
        alert: non

The current Hue software creates 'rooms', which are groups with a type value set to Room:

In [17]:
groups = bridge.groups()
rooms = [(gid, info['name']) for gid, info in groups.items() if info.get('type') == 'Room' ]
for room_id, info in rooms:
    print "{:3} : {}".format(room_id, info)

5   : Garden
4   : Kitchen
7   : Hall
6   : Sitting room
9   : Bedroom
8   : Landing


## Sensors

Sensors are mostly switches, but a few other things come under the same category in the bridge.  There's a 'daylight' sensor, implemented in software, for example, and various bits of state can also be stored here so they can be used in rule conditions later.

In [20]:
sensors = bridge.sensors()
summary = [(info['name'], i, info['type']) for i,info in sensors.items()]
summary.sort(lambda a,b: cmp(a[0], b[0]))  # Sort by name
for n,i,t in summary:
    print "{:30} {:>3} {}".format(n,i,t) 
    #print bridge.sensors[i]()
    

Bedroom tap                      7 ZGPSwitch
Daylight                         1 Daylight
Dimmer Switch 11 SceneCycle     14 CLIPGenericStatus
Dimmer Switch 12 SceneCycle     13 CLIPGenericStatus
Dining Room                      9 ZGPSwitch
Hall                             8 ZGPSwitch
Hall dimmer                     12 ZLLSwitch
Kitchen dimmer                  11 ZLLSwitch
Kitchen tap                      2 ZGPSwitch
Landing tap                      3 ZGPSwitch
Laundry tap                      4 ZGPSwitch
Sitting room                    10 ZGPSwitch
Top Tap                          6 ZGPSwitch


Here's a more complete list:

In [19]:
print yaml.safe_dump(bridge.sensors(), indent=4)

'1':
    config: {lat: none, long: none, 'on': true, sunriseoffset: 30, sunsetoffset: -30}
    manufacturername: Philips
    modelid: PHDL00
    name: Daylight
    state: {daylight: null, lastupdated: none}
    swversion: '1.0'
    type: Daylight
'10':
    config: {'on': true}
    manufacturername: Philips
    modelid: ZGPSWITCH
    name: Sitting room
    state: {buttonevent: 34, lastupdated: '2016-08-07T10:20:04'}
    type: ZGPSwitch
    uniqueid: 00:00:00:00:00:41:1f:34-f2
'11':
    config: {battery: 100, 'on': true, reachable: true}
    manufacturername: Philips
    modelid: RWL021
    name: Kitchen dimmer
    state: {buttonevent: null, lastupdated: none}
    swversion: 5.45.1.17846
    type: ZLLSwitch
    uniqueid: 00:17:88:01:10:33:28:66-02-fc00
'12':
    config: {battery: 100, 'on': true, reachable: true}
    manufacturername: Philips
    modelid: RWL021
    name: Hall dimmer
    state: {buttonevent: 1002, lastupdated: '2016-08-06T22:50:32'}
    swversion: 5.45.1.17846
    type: 

## Rules

Rules map sensor events etc. to actions.


In [22]:
rules = bridge.rules()
print yaml.safe_dump(rules, indent=4)


'1':
    actions:
    -   address: /groups/4/action
        body: {scene: g7tDrSN6Hpofhfj}
        method: PUT
    conditions:
    - {address: /sensors/2/state/lastupdated, operator: dx}
    - {address: /sensors/2/state/buttonevent, operator: eq, value: '18'}
    created: '2016-04-29T08:22:45'
    lasttriggered: '2016-08-06T20:49:59'
    name: Tap 2.4
    owner: s4zx5V9P01QX2tXIQpntlzfzVHfLpBLscIGWgfqP
    recycle: false
    status: enabled
    timestriggered: 2
'10':
    actions:
    -   address: /groups/8/action
        body: {scene: zvuMOXo8vmShFZK}
        method: PUT
    conditions:
    - {address: /sensors/4/state/lastupdated, operator: dx}
    - {address: /sensors/4/state/buttonevent, operator: eq, value: '18'}
    created: '2016-07-23T11:48:50'
    lasttriggered: none
    name: Tap 4.4
    owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
    recycle: false
    status: enabled
    timestriggered: 0
'11':
    actions:
    -   address: /groups/8/action
        body: {scene: 2HASTwK

Show the rules triggered by the Sitting Room switch.

For Tap switches, buttons 1,2,3,4 are represented by the values 34,16,17,18 respectively.

In [23]:
switch = '10'  # sitting room
print "Switch {} -- {}\n".format(switch, sensors[switch]['name'])

# State changes on the switch will look like this:
state_string = "/sensors/{}/state/".format(switch)

# Look through the rules for once which contain this 
# string in their conditions:
for rid, info in rules.items():
    this_switch = False
    matching_conditions = [c for c in info['conditions'] if state_string in c['address']]
    if len(matching_conditions) > 0:
        print "{:3} {:20}".format(rid, info['name'])
        for c in info['conditions']:
            print "   ? condition {}".format(c)
        for a in info['actions']:

            # If the action involves applying a scene, get its name
            scene_name = ""
            if 'scene' in a['body']:
                scene_name = scenes[a['body']['scene']]['name']
            
            print "   - action address {} body {:16} {} ".format( a['address'], a['body'], scene_name)
            

Switch 10 -- Sitting room

29  Tap 10.3            
   ? condition {u'operator': u'dx', u'address': u'/sensors/10/state/lastupdated'}
   ? condition {u'operator': u'eq', u'value': u'17', u'address': u'/sensors/10/state/buttonevent'}
   - action address /groups/6/action body {u'scene': u'SUThT3XiV7sSzml'} Warm 
31  Tap 10.4            
   ? condition {u'operator': u'dx', u'address': u'/sensors/10/state/lastupdated'}
   ? condition {u'operator': u'eq', u'value': u'18', u'address': u'/sensors/10/state/buttonevent'}
   - action address /groups/6/action body {u'scene': u'3owQUn01W7nVsxR'} Evening 
30  Tap 10.2            
   ? condition {u'operator': u'dx', u'address': u'/sensors/10/state/lastupdated'}
   ? condition {u'operator': u'eq', u'value': u'16', u'address': u'/sensors/10/state/buttonevent'}
   - action address /groups/6/action body {u'scene': u'SePln7Lt9-H7Hm7'} Bright 
32  Tap 10.1            
   ? condition {u'operator': u'dx', u'address': u'/sensors/10/state/lastupdated'}
   ? c

Let's see what is actually done by one of these scenes:

In [189]:
scene='3owQUn01W7nVsxR' # 'Evening' scene button 10.4

s = bridge.scenes[scene]()
print yaml.safe_dump(s, indent=4)

appdata: {data: mdVDQ_r06_d99, version: 1}
lastupdated: '2016-08-07T09:36:40'
lights: ['14', '21', '24', '25']
lightstates:
    '14':
        bri: 189
        'on': true
        xy: [0.5102, 0.3642]
    '21': {bri: 189, 'on': true}
    '24': {bri: 128, ct: 200, 'on': true}
    '25': {bri: 128, ct: 200, 'on': true}
locked: true
name: Evening
owner: zeQomfNu-w-p1PLM8oeYTiXbtqsxn-q1-7MLIN4B
picture: ''
recycle: false
version: 2

